In [1]:
import pandas as pd
import numpy as np
import random
import string
import os

In [2]:
def duplicar_random(file_path, n):
    lines = open(F"{file_path}").readlines()
    lista_archivos = []
    for i in range(n):
        random.shuffle(lines)
        with open(F"../scripts/instancias/random/random-{i}.txt", 'w') as f:
            f.writelines(lines)
        lista_archivos.append(F"../scripts/instancias/random/random-{i}.txt")
    with open(F"instancias/random/indice.txt", 'w') as f:
        print("Indice archivos",file=f)
        for j in range(n):
            print(lista_archivos[j], file=f)

## Muchos vs pocos archivos

In [2]:
def juntar_en_un_archivo(source_directory_path,target_file_path):
    df_indice = pd.read_csv(source_directory_path + "/indice.txt")
    with open(target_file_path,'w') as target:
        for source_path in df_indice.stack().to_list():
            with open(source_path, 'r') as source:
                target.write(source.read())

In [63]:
def separar_en_archivos(source_file_path, n, target_directory_path):
    source_list = pd.read_csv(source_file_path).stack().to_list()
    output_line_count = len(source_list)//n
    current_line = 0
    lista_indice = []
    for i in range(n):
        target_filepath = target_directory_path +"/separado-"+str(i)+".txt"
        lista_indice.append(target_filepath)
        with open(target_filepath, 'w') as target:
            for j in range(output_line_count):
                try:
                    print(source_list[current_line], file=target)
                except IndexError:
                    print(F"fallo en la linea {current_line}/{len(source_list)} en el archivo Nro {i} durante la iteracion Nro {j}")
                    raise IndexError("list index out of range")
                current_line += 1
            #En el ultimo archivo metemos las lineas "que faltaban" por el redondeo
            if(i == n-1):
                while(current_line < len(source_list)):
                    print(source_list[current_line],file = target)
                    current_line += 1
    with open(target_directory_path+"/indice.txt",'w') as indice:
        print("Indice directorios",file=indice)
        for path in lista_indice:
            print(path,file=indice)


In [3]:
duplicar_random("../data/corpus", 100)

In [12]:
juntar_en_un_archivo(source_directory_path="instancias/random",target_file_path="instancias/separados/fuente.txt")

In [64]:
cant_archivos = [2,4,8,16,32,64,128,256,512,1024]
with open("instancias/separados/indice_directorios.txt", 'w') as indice:
    print("Indice directorios",file=indice)
    for cantidad in cant_archivos:
        path = F"instancias/separados/{cantidad}_archivos"
        print(path,file=indice)
        try: 
            os.mkdir(path)
        except FileExistsError: 
            pass
        separar_en_archivos("instancias/separados/fuente.txt",cantidad,path)    

## Dataset menos uniformes las palabras

In [3]:
tamanio = len(open("../data/corpus_original").readlines())

In [4]:
def archivoAleatorio(cantPalabras,filepath,alfabeto):
    with open(filepath,'w') as out:
        for _ in range(cantPalabras):
            palabra = ""
            for _ in range(random.randint(1,10)):
                palabra += random.choice(alfabeto)
            print(palabra,file=out)

In [5]:
def generarDatasetAleatorioNoUniforme(n,alfabeto,pathDirectorio,nombreDeArchivo):
    try:
        lista_archivos = []
        for i in range(n):
            archivoAleatorio(tamanio,F"{pathDirectorio}/{nombreDeArchivo}-{i}.txt",alfabeto)
            lista_archivos.append(F"{pathDirectorio}/{nombreDeArchivo}-{i}.txt")
        with open(F"{pathDirectorio}/indice.txt",'w') as indice:
            print("Indice",file=indice)
            for path in lista_archivos:
                print(path,file=indice)
    except FileNotFoundError:
        os.mkdir(pathDirectorio)
        generarDatasetAleatorioNoUniforme(n,alfabeto,pathDirectorio,nombreDeArchivo)


In [10]:
try: 
    os.mkdir("../scripts/instancias/randomNoUniforme")
except FileExistsError: 
    pass
generarDatasetAleatorioNoUniforme(50,string.ascii_lowercase,"../scripts/instancias/randomNoUniforme","randomNoUniforme")

## Distribuciones distintas

In [13]:
alfabeto1 = (string.ascii_lowercase)[:3]
alfabeto2 = (string.ascii_lowercase)[:13]
alfabeto3 = (string.ascii_lowercase)
alfabetos = [alfabeto1,alfabeto2,alfabeto3]

In [14]:
try: 
    os.mkdir("../scripts/instancias/multiplesAlfabetos")
except FileExistsError: 
    pass
lista_directorios = []
for i in range(len(alfabetos)):
    lista_directorios.append(F"../scripts/instancias/multiplesAlfabetos/alfabeto-{i}")
    generarDatasetAleatorioNoUniforme(50,alfabetos[i],F"../scripts/instancias/multiplesAlfabetos/alfabeto-{i}",F"alfabeto-{i}")
with open(F"../scripts/instancias/multiplesAlfabetos/indice.txt",'w') as indice:
    print("Indice",file=indice)
    for path in lista_directorios:
        print(path,file=indice)